### Test case: calculating H_nn for a bath with 2 nuclei with I=0.5. 

For two nuclei: we use the following basis:

$\begin{bmatrix}
\uparrow \uparrow
\\ \uparrow \downarrow
\\ \downarrow \uparrow
\\ \downarrow \downarrow
\end{bmatrix}$

Evidently, $\hat{I_z}$ matrix takes the form of 

$$\hat{I_z} =  \hat{\sigma_z} \otimes \mathbb{1}  + \mathbb{1} \otimes \hat{\sigma_z},$$

$$\sigma_z = \begin{bmatrix}
1 & 0\\
0 & -1 
\end{bmatrix}$$


For general nuclear spin operator in 3D: 
$$\hat{I_1} = \vec{\sigma} \otimes \mathbb{1}$$
$$\hat{I_2} = \mathbb{1} \otimes \vec{\sigma},$$

and

$$\vec{\sigma} = \hat{\sigma_x} + \hat{\sigma_y} + \hat{\sigma_z}$$

below I attempt to write the nuclear-nuclear interaction part of the bath hamiltonian

$$H_{nn} = \frac{\mu_0}{4\pi}\sum_{i<j}{}\gamma_i\gamma_j\hbar^2(\frac{\vec{I_i}\cdot\vec{I_j}}{r_{ij}^{3}} - \frac{3(\vec{I_i} \cdot \vec{r_{ij}})(\vec{I_j} \cdot \vec{r_{ij}})}{r_{ij}^{5}})$$




In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from qutip import *

B = [0,0,1] #magnetic field (unused now)
r1=[3,4,5] #position of n1
r2=r1+[5,5,5]/np.sqrt(3) #position of n2
r=r2-r1
c=1 #constant
I_array=[0.5,0.5] # nuclear spin of spin 1 and 2


I1=np.array([tensor(spin_Jx(I_array[0]),qeye(2)), #use J spin matrix in case spin !=1/2 
             tensor(spin_Jy(I_array[0]),qeye(2)),
             tensor(spin_Jz(I_array[0]),qeye(2))], 
            dtype=object)
I2=np.array([tensor(qeye(2),spin_Jx(I_array[1])),
             tensor(qeye(2),spin_Jy(I_array[1])),
             tensor(qeye(2),spin_Jz(I_array[1]))],
            dtype=object)

H_nn = c*(I1@I2/np.linalg.norm(r)**3 - 3*(I1@r)*(I2@r)/np.linalg.norm(r)**5)

In [2]:
H_nn

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.   +0.j    -0.002+0.002j -0.002+0.002j  0.   +0.004j]
 [-0.002-0.002j  0.   +0.j     0.   +0.j     0.002-0.002j]
 [-0.002-0.002j  0.   +0.j     0.   +0.j     0.002-0.002j]
 [ 0.   -0.004j  0.002+0.002j  0.002+0.002j  0.   +0.j   ]]